<a href="https://colab.research.google.com/github/BOLLOJUAISHWARYA/Google-collab-ANN/blob/main/stand_with_gridsearchcv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LOAN PREDICTION 

Problem statement: Need to predict if the given person will get loan or not based on the given variables

Inference : Binary classification problem





# **steps performed in this sheet are**

implementing gridsearch with batchsize ,epochs and neurons to find which combination gives the best accuracy




In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

Connecting to google drive to import data inserted the code snippet 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the csv file using pandas 


In [ ]:
traindata=pd.read_csv('/content/drive/MyDrive/Datasets/loan predictor/train/train.csv')

unique is used to display the unique values

In [ ]:
# Dropping loan_id
traindata.drop('Loan_ID', axis=1 , inplace=True)

In [ ]:
print(traindata['Credit_History'].unique())
print(traindata['Dependents'].unique())
print(traindata['Property_Area'].unique())


[ 1.  0. nan]
['0' '1' '2' '3+' nan]
['Urban' 'Rural' 'Semiurban']


'Gender', 'Married','Education', 'Self_Employed', 'Property_Area'--categorical

for categorical missing values replaced with mode


In [ ]:
traindata['Gender'].fillna(traindata['Gender'].mode()[0],inplace=True)
traindata['Married'].fillna(traindata['Married'].mode()[0],inplace=True)
traindata['Education'].fillna(traindata['Education'].mode()[0],inplace=True)
traindata['Self_Employed'].fillna(traindata['Self_Employed'].mode()[0],inplace=True)
traindata['Property_Area'].fillna(traindata['Property_Area'].mode()[0],inplace=True)
traindata['Dependents'].fillna(traindata['Dependents'].mode()[0],inplace=True)

For numerical missing values replaced with mean

In [ ]:
traindata['LoanAmount'].fillna(traindata['LoanAmount'].mean(),inplace=True)
traindata['Loan_Amount_Term'].fillna(traindata['Loan_Amount_Term'].mean(),inplace=True)
traindata['Credit_History'].fillna(traindata['Credit_History'].mean(),inplace=True)

Manual encoding

In [ ]:
traindata['Gender'].replace({'Male':1,'Female':0},inplace=True)
traindata['Education'].replace({'Graduate':1,'Not Graduate':0},inplace=True)
traindata['Property_Area'].replace({'Rural':0,'Semiurban':1,'Urban':2},inplace=True)
traindata['Self_Employed'].replace({'Yes':1,'No':0},inplace=True)
traindata['Married'].replace({'Yes':1,'No':0},inplace=True)
traindata['Dependents'].replace({'3+':3},inplace=True)

In [ ]:
traindata.dtypes

Gender                 int64
Married                int64
Dependents            object
Education              int64
Self_Employed          int64
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int64
Loan_Status           object
dtype: object

In [ ]:
traindata['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

In [ ]:
continuous = ['LoanAmount','Loan_Amount_Term','Credit_History','CoapplicantIncome']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

for var in continuous:
    traindata[var] = traindata[var].astype('float64')
    traindata[var] = scaler.fit_transform(traindata[var].values.reshape(-1, 1))

traindata['ApplicantIncome'] = scaler.fit_transform(traindata['ApplicantIncome'].values.reshape(-1, 1))

In [ ]:
traindata.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,1,0,0.072991,-0.554487,0.000000,0.279851,0.45164,2,1
1,1,1,1,1,0,-0.134412,-0.038732,-0.219273,0.279851,0.45164,0,0
2,1,1,0,1,1,-0.393747,-0.554487,-0.957641,0.279851,0.45164,2,1
3,1,1,0,0,0,-0.462062,0.251980,-0.314547,0.279851,0.45164,2,1
4,1,0,0,1,0,0.097728,-0.554487,-0.064454,0.279851,0.45164,2,1


In [ ]:
traindata=traindata.astype(float)

In [ ]:
from sklearn.model_selection import train_test_split
X = traindata.iloc[:,:-1].values
Y = traindata.iloc[:,-1].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

KerasClassifier wrapper can take arguments, We can use these arguments to further customize the construction of the model.

Importing Gridsearch for hyperparameter tuning to get best parameters for our model. 

here we have used epoch,batch-size and number of neurons for hyperparameter tuning

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
def build_clf(unit):
  # creating the layers of the NN
  classmodel = tf.keras.models.Sequential()
  classmodel.add(tf.keras.layers.Dense(units=unit, activation='relu'))
  classmodel .add(tf.keras.layers.Dense(1, activation='sigmoid'))
  classmodel .compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return classmodel 
  
model=KerasClassifier(build_fn=build_clf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
params={'batch_size':[32,16], 
        'nb_epoch':[100,200,300],     
        'unit':[12] 
        }
gs=GridSearchCV(estimator=model, param_grid=params, cv=10)

gs = gs.fit(X_train, Y_train)

31/31 [==============================] - 0s 2ms/step - loss: 0.6106 - accuracy: 0.7189


In [ ]:
best_params=gs.best_params_
accuracy=gs.best_score_

print(best_params,accuracy)

{'batch_size': 16, 'nb_epoch': 100, 'unit': 12} 0.6435510277748108
